In [ ]:
import apache_log_parser
from rich.progress import track
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
line_parser = apache_log_parser.make_parser("%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-Agent}i\"")

log_file = open('data/access.log', 'r')
log = []

log = [line for line in log_file]

log_file.close()

parsed_log = [line_parser(line) for line in track(log)]

In [ ]:
df = pd.DataFrame(parsed_log)

small_df = df[['remote_host', 'request_url_path', 'time_received_datetimeobj', 'response_bytes_clf']].dropna()
small_df['response_bytes_clf'] = pd.to_numeric(small_df['response_bytes_clf'])

In [ ]:
filtered_small_df = small_df[~small_df['request_url_path'].str.contains('\.')]
endpoint_list = ('/trail_rankings', '/rankings', '/search', '/about', '/map', '/interactive-map', '/explore')
filtered_small_df = filtered_small_df[filtered_small_df['request_url_path'].str.startswith(endpoint_list)]

In [ ]:
filtered_small_df.describe()

In [ ]:
len(filtered_small_df.remote_host.unique())

In [ ]:
filtered_small_df.remote_host.value_counts().head(50)

In [ ]:
filtered_small_df['date'] = pd.to_datetime(filtered_small_df['time_received_datetimeobj']).dt.date

In [ ]:
spam_df = filtered_small_df[filtered_small_df['remote_host'] == '73.17.249.14']
spam_df.head(10)

In [ ]:
trimmed_df = filtered_small_df[filtered_small_df['date'] > pd.to_datetime('2023-03-01').date()]

In [ ]:
first_date = trimmed_df['date'].head(1)
last_date = trimmed_df['date'].tail(1)

In [ ]:
month_diff = ((last_date.to_list()[0].year * 12) + (last_date.to_list()[0].month)) - ((first_date.to_list()[0].year * 12) + (first_date.to_list()[0].month))

In [ ]:
plt.hist(trimmed_df['date'], bins=month_diff)
plt.xticks(rotation=90)

In [ ]:
trimmed_df['date'].hist()

In [ ]:
trimmed_df[['response_bytes_clf', 'date']].groupby(['date']).sum().plot()

In [ ]:
import user_db

In [ ]:
conn = user_db.tuple_cursor()

log_df = pd.read_sql_query("SELECT * FROM Log", conn)

In [ ]:
len(log_df)

In [ ]:
query = 'SELECT COUNT(DISTINCT ip) FROM Log WHERE NOT page_category = "/"'
print(conn.execute(query).fetchall())
conn.close()